In [4]:
# input params:
# opco[string]
# start_datetime[string] - ex: 1900-01-01 00:00:00
# end_datetime[string] - ex: 2026-02-09 06:45:00 ## In current implementation No end_datetime, they are filtering based on last run time, any data after that 
# run_datetime[string]
# [change timezone of emr]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "com.databricks:spark-xml_2.12:0.18.0"
    }
}


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.getOrCreate()
#.config("spark.jars.packages","com.databricks:spark-xml_2.12:0.18.0") \


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1771589533394_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark.conf.get("spark.jars.packages")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'com.databricks:spark-xml_2.12:0.18.0'

In [14]:
from pyspark.sql import functions as f

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# define input params
opco="oh"
start_datetime=""
end_datetime=""
run_datetime="20260219_101235"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# def get_filtered_s3_paths(bucket, base_prefix, target_timestamp):
def get_filtered_s3_paths():
    """
    Lists S3 prefixes and filters for those >= target_timestamp.
    
    """
    return \
    ["s3://aep-datalake-raw-dev/util/intervals/nonvee/uiq_auth/oh/202602/20260201_0431/20260201-1a0c8939-446c-41e6-b427-8f79d9205403-1-72.xml.gz"
    , "s3://aep-datalake-raw-dev/util/intervals/nonvee/uiq_auth/oh/202602/20260201_0431/20260201-1a0c8939-446c-41e6-b427-8f79d9205403-1-73.xml.gz"
    , "s3://aep-datalake-raw-dev/util/intervals/nonvee/uiq_auth/oh/202602/20260201_0431/20260201-1a0c8939-446c-41e6-b427-8f79d9205403-1-74.xml.gz"
    ]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
paths = ",".join(get_filtered_s3_paths())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# df = spark.read\
# .format("com.databricks.spark.xml")\
# .option("inferSchema", "true")\
# .option("rowTag", "MeterData")\
# .option("rootTag", "MeterData")\
# .option("valueTag", "_text_content")\ # _value 
# .load(paths)


df = (spark.read
.format("xml")
.option("rowTag", "MeterData")
.option("attributePrefix", "_") #\ # XML attributes get _ prefix
.option("valueTag", "VALUE") #\ # text node content tag
.option("nullValue", "")
.option("mode", "PERMISSIVE")
.option("inferSchema", "true")
# .schema(schema)
.load(paths))



# .withColumnRenamed()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- IntervalReadData: struct (nullable = true)
 |    |-- Interval: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- Reading: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- _BlockEndValue: double (nullable = true)
 |    |    |    |    |    |-- _Channel: long (nullable = true)
 |    |    |    |    |    |-- _RawValue: double (nullable = true)
 |    |    |    |    |    |-- _UOM: string (nullable = true)
 |    |    |    |    |    |-- _Value: double (nullable = true)
 |    |    |    |    |    |-- _text_content: string (nullable = true)
 |    |    |    |-- _BlockSequenceNumber: long (nullable = true)
 |    |    |    |-- _EndTime: timestamp (nullable = true)
 |    |    |    |-- _GatewayCollectedTime: timestamp (nullable = true)
 |    |    |    |-- _IntervalSequenceNumber: long (nullable = true)
 |    |-- _EndTime: timestamp (nullable = true)
 |    |-- _IntervalLength: lon

In [9]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- IntervalReadData: struct (nullable = true)
 |    |-- Interval: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- Reading: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- VALUE: string (nullable = true)
 |    |    |    |    |    |-- _BlockEndValue: double (nullable = true)
 |    |    |    |    |    |-- _Channel: long (nullable = true)
 |    |    |    |    |    |-- _RawValue: double (nullable = true)
 |    |    |    |    |    |-- _UOM: string (nullable = true)
 |    |    |    |    |    |-- _Value: double (nullable = true)
 |    |    |    |-- _BlockSequenceNumber: long (nullable = true)
 |    |    |    |-- _EndTime: timestamp (nullable = true)
 |    |    |    |-- _GatewayCollectedTime: timestamp (nullable = true)
 |    |    |    |-- _IntervalSequenceNumber: long (nullable = true)
 |    |-- _EndTime: timestamp (nullable = true)
 |    |-- _IntervalLength: long (nulla

In [19]:
df.select(
    f.col("IntervalReadData._IntervalLength").alias("intervallength")
    , f.col("IntervalReadData._EndTime").alias("int_endtime")
    , f.col("IntervalReadData.Interval._GatewayCollectedTime").alias("int_gatewaycollectedtime")
).show(1, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+-----------------+-------------------+--------------------+
|    IntervalReadData|              _MacID|_MeterName|    _UtilDeviceID|_UtilServicePointID|              _xmlns|
+--------------------+--------------------+----------+-----------------+-------------------+--------------------+
|{[{[{52697.4289, ...|00:13:50:05:ff:21...| 884393139|1ND884393139NMD06|     10720306603931|urn:com:ssn:schem...|
+--------------------+--------------------+----------+-----------------+-------------------+--------------------+
only showing top 1 row

## Read with schema

In [8]:
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
authoritative_data_schema = StructType([
StructField("_MacID", StringType(), True),
StructField("_MeterName", LongType(), True),
StructField("_UtilDeviceID", StringType(), True),
StructField("_UtilServicePointID", LongType(), True),
# StructField("_xmlns", StringType(), True),

StructField("IntervalReadData", StructType([
    StructField("_EndTime", TimestampType(), True),
    StructField("_IntervalLength", LongType(), True),
    StructField("_LpSetId", LongType(), True),
    StructField("_NumberIntervals", LongType(), True),
    StructField("_StartTime", TimestampType(), True),
    
    StructField("Interval", ArrayType(
        StructType([
            StructField("_BlockSequenceNumber", LongType(), True),
            StructField("_EndTime", TimestampType(), True),
            StructField("_GatewayCollectedTime", TimestampType(), True),
            StructField("_IntervalSequenceNumber", LongType(), True),
            
            StructField("Reading", ArrayType(
                StructType([
                    StructField("_BlockEndValue", DoubleType(), True),
                    StructField("_Channel", LongType(), True),
                    StructField("_RawValue", DoubleType(), True),
                    StructField("_UOM", StringType(), True),
                    StructField("_Value", DoubleType(), True),
                    StructField("_text_content", StringType(), True)
                ]), True)
            )
        ]), True
    ))
]))

])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df = spark.read\
.format("com.databricks.spark.xml")\
.option("rowTag", "MeterData")\
.option("rootTag", "MeterData")\
.option("valueTag", "_text_content")\
.schema(authoritative_data_schema)\
.load(paths)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _MacID: string (nullable = true)
 |-- _MeterName: long (nullable = true)
 |-- _UtilDeviceID: string (nullable = true)
 |-- _UtilServicePointID: long (nullable = true)
 |-- IntervalReadData: struct (nullable = true)
 |    |-- _EndTime: timestamp (nullable = true)
 |    |-- _IntervalLength: long (nullable = true)
 |    |-- _LpSetId: long (nullable = true)
 |    |-- _NumberIntervals: long (nullable = true)
 |    |-- _StartTime: timestamp (nullable = true)
 |    |-- Interval: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _BlockSequenceNumber: long (nullable = true)
 |    |    |    |-- _EndTime: timestamp (nullable = true)
 |    |    |    |-- _GatewayCollectedTime: timestamp (nullable = true)
 |    |    |    |-- _IntervalSequenceNumber: long (nullable = true)
 |    |    |    |-- Reading: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- _BlockEndValue: double (null

In [15]:
df = (
    df
.withColumnRenamed("_MeterName", "metername")
.withColumnRenamed("_UtilDeviceID", "utildeviceid")
.withColumnRenamed("_MacID", "macid")
.withColumnRenamed("IntervalReadData", "interval_reading")
.withColumnRenamed("_UtilServicePointID", "utilservicepointid") 
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- macid: string (nullable = true)
 |-- metername: long (nullable = true)
 |-- utildeviceid: string (nullable = true)
 |-- utilservicepointid: long (nullable = true)
 |-- interval_reading: struct (nullable = true)
 |    |-- _EndTime: timestamp (nullable = true)
 |    |-- _IntervalLength: long (nullable = true)
 |    |-- _LpSetId: long (nullable = true)
 |    |-- _NumberIntervals: long (nullable = true)
 |    |-- _StartTime: timestamp (nullable = true)
 |    |-- Interval: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _BlockSequenceNumber: long (nullable = true)
 |    |    |    |-- _EndTime: timestamp (nullable = true)
 |    |    |    |-- _GatewayCollectedTime: timestamp (nullable = true)
 |    |    |    |-- _IntervalSequenceNumber: long (nullable = true)
 |    |    |    |-- Reading: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- _BlockEndValue: double (nullable

In [18]:
df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-----------------+------------------+--------------------+
|               macid|metername|     utildeviceid|utilservicepointid|    interval_reading|
+--------------------+---------+-----------------+------------------+--------------------+
|00:13:50:05:ff:1f...|687054561|J3D687054561PMNA8|     7593957000011|{2026-02-01 05:00...|
+--------------------+---------+-----------------+------------------+--------------------+
only showing top 1 row

# 4-auth_feed_stg_load.sh

## Load of main daily staging table from source (`stg_nonvee.authorative_data_{opco}_stg`)
dml: `auth_interval_data_src_to_stg.dml`

In [24]:
# ============================================================
# CTE 1: intervals_exploded_lat_vws
# ============================================================

intervals_exploded_df = (
    df.select(
        f.col("_MeterName").alias("metername")
        , f.col("IntervalReadData._IntervalLength").alias("intervallength")
        , f.col("IntervalReadData.Interval").alias("interval_arr")
    )
    # LATERAL VIEW explode(outer_interval_read_data.intervalreaddata)
    # -> explode Interval array (middle level)
    .withColumn("middle_interval", f.explode(f.col("interval_arr")))
    .select(
        f.col("metername"),
        f.col("intervallength"),
        f.col("middle_interval._EndTime").alias("int_endtime"),
        f.col("middle_interval._GatewayCollectedTime").alias("int_gatewaycollectedtime"),
        f.col("middle_interval.Reading").alias("reading_arr")
    )

    # LATERAL VIEW explode(middle_interval.interval)
    # -> explode Reading array (inner level)
    .withColumn("inner_reading", f.explode(f.col("reading_arr")))
    .select(
        f.col("metername"),
        f.col("intervallength"),
        f.col("int_endtime"),
        f.col("int_gatewaycollectedtime"),
        f.col("inner_reading._Channel").alias("channel"),
        f.col("inner_reading._RawValue").alias("rawvalue"),
        f.col("inner_reading._Value").alias("value"),
        f.col("inner_reading._UOM").alias("uom")
    )
    
    # WHERE clause — drop garbage/incomplete records
    .filter(
        f.col("channel").isNotNull() &
        f.col("value").isNotNull() &
        f.col("uom").isNotNull()
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
intervals_exploded_df.show(n=10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------+-------------------+------------------------+-------+--------+-----+--------+
|metername|intervallength|int_endtime        |int_gatewaycollectedtime|channel|rawvalue|value|uom     |
+---------+--------------+-------------------+------------------------+-------+--------+-----+--------+
|687054561|15            |2026-01-31 05:15:00|2026-01-31 06:10:25.334 |101    |2.0     |0.006|kWh     |
|687054561|15            |2026-01-31 05:15:00|2026-01-31 06:10:25.334 |102    |0.0     |0.0  |kWh(rec)|
|687054561|15            |2026-01-31 05:15:00|2026-01-31 06:10:25.334 |108    |2.0     |0.006|kVARh   |
|687054561|15            |2026-01-31 05:15:00|2026-01-31 06:10:25.334 |122    |1200.0  |120.0|VrmsA-N |
|687054561|15            |2026-01-31 05:15:00|2026-01-31 06:10:25.334 |123    |1226.0  |122.6|VrmsB-N |
|687054561|15            |2026-01-31 05:15:00|2026-01-31 06:10:25.334 |124    |1286.0  |128.6|VrmsC-N |
|687054561|15            |2026-01-31 05:15:00|2026-01-31 06:10:2

#### usage_nonvee.uom_mapping ~~not available in~~ dev

In [28]:
# ============================================================
# UOM Mapping — LEFT JOIN subquery
# ============================================================
uom_mapping_df = (
    spark.table("usage_nonvee.uom_mapping")
    .filter(f.col("aep_opco") == opco)
    .select(
        f.col("aep_opco"),
        f.col("aep_channel_id"),
        f.col("aep_derived_uom"),
        f.col("name_register"),
        f.col("aep_srvc_qlty_idntfr")
    )
    .distinct()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[TABLE_OR_VIEW_NOT_FOUND] The table or view `usage_nonvee`.`uom_mapping` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [usage_nonvee, uom_mapping], [], false

Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1771589533394_0001/container_1771589533394_0001_01_000001/pyspark.zip/pyspark/sql/session.py", line 1667, in table
    return DataFrame(self._jsparkSession.table(tableName), self)
  File "/mnt/yarn/usercache/livy/appcache/application_1771589533394_0001/container_1771589533394_0001_01_000001/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/mnt/yarn/usercache/livy/appcache/application_1

In [ ]:
# ============================================================
# CTE 2: formatted_times
# ============================================================

# Helper: clean endtime to 19 chars → "yyyy-MM-dd'T'HH:mm:ss"
# Hive: substr(int_endtime, 0, 19) — Hive substr is 0-indexed
# PySpark: substr(col, 1, 19) — PySpark substr is 1-indexed
clean_endtime = substr(col("int_endtime"), 1, 19)

# Hive: SUBSTR(int_endtime, -6, 6) — last 6 chars (timezone offset e.g. -05:00)
# PySpark: substr(col, -6, 6) — same behavior
interval_epoch = substr(col("int_endtime"), -6, 6)

formatted_times_df = (
    intervals_exploded_df
    .withColumn(
        "starttime",
        f.from_unixtime(
            f.unix_timestamp(clean_endtime, "yyyy-MM-dd'T'HH:mm:ss")
            - (f.cast(f.col("intervallength"), "int") * 60)
            , "yyyy-MM-dd'T'HH:mm:ss"
        )
    )
    .withColumn(
        "endtime",
        f.from_unixtime(
            f.unix_timestamp(clean_endtime, "yyyy-MM-dd'T'HH:mm:ss")
            , "yyyy-MM-dd'T'HH:mm:ss"
        )
    )
    .withColumn("interval_epoch", interval_epoch)
     # LEFT JOIN uom_mapping on channel = aep_channel_id
    .join(uom_mapping, f.col("channel") == f.col("aep_channel_id"), "left")
    .select(
        f.col("metername"),
        f.col("intervallength"),
        f.col("starttime"),
        f.col("endtime"),
        f.col("interval_epoch"),
        f.col("int_gatewaycollectedtime"),
        f.col("channel"),
        f.col("rawvalue"),
        f.col("value"),
        f.col("uom"),
        f.col("aep_derived_uom").alias("uom_aep_derived_uom")
    )
)

In [ ]:
# ==============================================================================
# Load data into daily staging table (stg_nonvee.authorative_data_{opco}_stg)`
# ==============================================================================
authorative_data_opco_stg = (
    formatted_times_df
    .filter(
    # Hive substr(starttime, 0, 10) = PySpark substr(col, 1, 10) — index offset
        f.substr(f.col("starttime"), 1, 10) > f.lit("2020-01-01")
    )
    .select(
        f.trim(f.col("metername")).alias("serialnumber"),
        f.col("intervallength"),
        f.concat(f.col("starttime"), f.col("interval_epoch")).alias("starttimeperiod"),
        f.concat(f.col("endtime"), f.col("interval_epoch")).alias("endtimeperiod"),
        f.col("interval_epoch"),
        f.col("int_gatewaycollectedtime"),
        f.lower(f.trim(f.col("uom"))).alias("aep_raw_uom"),
        f.col("rawvalue").alias("aep_raw_value"),
        f.col("value"),
        f.lit(opco).alias("aep_opco"), # '${hivevar:opco}'
        f.col("channel").alias("aep_channel_id"),
        f.substr(f.col("starttime"), 1, 10).alias("aep_usage_dt"),# substr 0->1 index fix
        f.coalesce(f.col("uom_aep_derived_uom"), f.lit("UNK")).alias("aep_derived_uom"), # nvl() equivalent
        f.substr(f.trim(f.col("metername")), -2, 2).alias("aep_meter_bucket")
    )
)


In [29]:
from datetime import datetime

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
day_name = datetime.strptime(run_datetime, "%Y%m%d_%H%M%S").strftime("%A")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
day_name

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Thursday'

In [ ]:
#Arguments
# ============================================================

weekly_stg_table = f"stg_nonvee.authoritative_data_{opco}weekly_stg"
# the below path is direct equivalent from hdfs 
# s3_weekly_stg = f"s3://aep-datalake-transform-{aws_env}/util/intervals/authoritative/holding/{opco}/authoritative_data{opco}_weekly_stg"
s3_weekly_stg
# ============================================================
# Weekend condition — mirrors shell script if/else
# ============================================================

is_weekend_processing = (
(day_name == "Sunday" and opco in ("oh", "im", "swp")) or
(day_name == "Monday" and opco in ("pso", "ap"))
)
# ============================================================
# Cutoff dates — reused in both branches
# ============================================================

cutoff_15d = date_format(date_sub(current_date(), 15), "yyyy-MM-dd")
cutoff_185d = date_format(date_sub(current_date(), 185), "yyyy-MM-dd")
run_dt = date_format(current_date(), "yyyyMMdd")

# ============================================================
# BRANCH A: Weekend Processing
# auth_interval_data_weekly_hold_to_stg
# ============================================================

if is_weekend_processing:
logger.info(f"Weekend processing triggered for opco={opco} on {day_name}")

# --- Step 1: Read weekly_stg from Glue/S3 ---
logger.info(f"Reading weekly staging table: {weekly_stg_table}")
weekly_stg_df = spark.read.table(weekly_stg_table)

# Row counts for audit (mirrors Hive count queries)
logger.info(f"Weekly STG row count: {weekly_stg_df.count()}")

# --- Dedup via row_number (mirrors WITH weekly_hold CTE) ---
window_spec = Window.partitionBy(
    "serialnumber",
    "endtimeperiod",
    "aep_channel_id",
    "aep_raw_uom"
).orderBy(col("int_gatewaycollectedtime").desc())

deduped_df = (
    weekly_stg_df
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn", "run_dt")               # drop partition col before appending to stg
)

# --- INSERT INTO stg_df (append deduped weekly records) ---
# stg_df already exists in memory from earlier in the pipeline
stg_df = stg_df.union(deduped_df)
logger.info(f"STG row count after weekly merge: {stg_df.count()}")

# --- Step 2: Drop all partitions from weekly_stg Glue table ---
# Equivalent to: ALTER TABLE ... DROP PARTITION (run_dt>0)
logger.info(f"Dropping all partitions from {weekly_stg_table}")
spark.sql(f"ALTER TABLE {weekly_stg_table} DROP IF EXISTS PARTITION (run_dt > '0')")

# --- Step 3: Remove S3 data for weekly_stg ---
logger.info(f"Removing S3 data from {s3_weekly_stg}")
sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
    sc._jvm.java.net.URI.create(s3_weekly_stg),
    hadoop_conf
)
path = sc._jvm.org.apache.hadoop.fs.Path(s3_weekly_stg)
if fs.exists(path):
    fs.delete(path, True)               # True = recursive delete
    logger.info("S3 weekly staging data removed successfully")
else:
    logger.info("S3 path does not exist, skipping delete")

============================================================
BRANCH B: Weekday — Hold old records to weekly_stg
auth_interval_data_hold_stg_to_weekly
============================================================

else:
logger.info(f"Weekday processing — holding old data for opco={opco} on {day_name}")

logger.info(f"STG row count before weekly hold: {stg_df.count()}")

# --- SQL-1: Records between 185d and 15d → append to weekly_stg (Glue table) ---
records_to_hold = (
    stg_df
    .filter(
        (col("aep_usage_dt") >= cutoff_185d) &
        (col("aep_usage_dt") <  cutoff_15d)         # BETWEEN 185d AND 15d
    )
    .withColumn("run_dt", run_dt)                    # partition column
)

logger.info(f"Records moving to weekly hold: {records_to_hold.count()}")

# Persist to Glue table partitioned by run_dt
records_to_hold.write \
    .mode("append") \
    .partitionBy("run_dt") \
    .format("parquet") \
    .saveAsTable(weekly_stg_table)

# Audit: run_dt + aep_usage_dt counts (mirrors Hive select at end of DML)
spark.read.table(weekly_stg_table) \
    .groupBy("run_dt", "aep_usage_dt") \
    .count() \
    .orderBy("run_dt", "aep_usage_dt") \
    .show()

# --- SQL-2: Keep only records >= 15d in stg_df (overwrite) ---
stg_df = (
    stg_df
    .filter(col("aep_usage_dt") >= cutoff_15d)
)

logger.info(f"STG row count after weekly hold: {stg_df.count()}")


In [ ]:
#1. generate base prefixes from opco
#2. generate list of files with modification time between start and end datetime x-->  sub-directories (yyyymm --> yyyymmdd_HHMM ) in base prefix
#3. read a saprk df --> input is above list